# Initial Data Import and Engineering

In [1]:
import pandas as pd

#### Import each csv file, rename columns, and add new columns for daily values where necessary

In [2]:
df_testing = pd.read_csv('statewide_testing.csv', parse_dates=['date'])
df_testing.set_index('date', inplace=True)
df_testing.rename(columns={'tested':'cumulative_tests'},inplace=True)
df_testing['new_tests'] = df_testing['cumulative_tests']
for i in range(1, len(df_testing)):
    df_testing.iloc[[i],[1]] = df_testing.iloc[i,0] - df_testing.iloc[(i-1),0]

In [23]:
df_cases = pd.read_csv('statewide_cases.csv', header=1, parse_dates=['date'])
df_cases = df_cases.groupby('date').sum()
df_cases.rename(columns={'totalcountconfirmed':'cumulative_cases','totalcountdeaths':'cumulative_deaths','newcountconfirmed':'new_cases','newcountdeaths':'new_deaths'}, inplace=True)

In [24]:
df_hospitals = pd.read_csv('hospitals_by_county.csv', header=1, parse_dates=['todays_date'])
df_hospitals = df_hospitals.groupby('todays_date').sum()
df_hospitals.drop(labels=['hospitalized_covid_patients','all_hospital_beds'], axis=1, inplace=True)

In [3]:
df_nursinghome = pd.read_csv('covid19datanursinghome.csv', header=1, parse_dates=['as_of_date'])
df_nursinghome.drop(labels=['facility_id'], axis=1, inplace=True)
df_nursinghome = df_nursinghome.groupby('as_of_date').sum()
df_nursinghome['new_resident_deaths'] = None
for i in range(1, len(df_nursinghome)):
    df_nursinghome.iloc[[i],[6]] = df_nursinghome.iloc[i,2] - df_nursinghome.iloc[(i-1),2]
nh_headers = {}
for item in df_nursinghome.columns:
    new_name = 'nh_' + item
    nh_headers.update({item:new_name})
df_nursinghome.rename(columns=nh_headers,inplace=True)

In [4]:
df_beds = pd.read_csv('bed_surge.csv', header=1, parse_dates=['date'])
df_beds = df_beds.groupby('date').sum()

#### Restructured age dataset to only include those in the high risk category (65+)

In [27]:
df_case_age = pd.read_csv('case_demographics_age.csv', header=1, parse_dates=['date'])
df_case_age.set_index('age_group', inplace=True)
df_case_age.drop(labels=['0-17','18-49','50-64','Unknown','Missing'], axis=0, inplace=True)
df_case_age.reset_index(inplace=True)
df_case_age.sort_values(by='date', inplace=True)
df_case_age.set_index('date', inplace=True)
df_case_age.drop(['deaths','ca_percent','age_group','case_percent'], axis=1, inplace=True)
df_case_age['elderly_new_cases'] = None
for i in range(1, len(df_case_age)):
    df_case_age.iloc[[i],[2]] = df_case_age.iloc[i,0] - df_case_age.iloc[(i-1),0]
df_case_age['elderly_new_cases'] = pd.to_numeric(df_case_age.elderly_new_cases)
df_case_age.rename(columns={'totalpositive':'elderly_cumulative_cases','deaths_percent':'elderly_proportion_deaths'}, inplace=True)

#### Restructured ethnicity dataset to isolate cases attributed to minority groups

In [28]:
df_case_ethnicity = pd.read_csv('case_demographics_ethnicity.csv', header=1, parse_dates=['date'])
df_case_ethnicity.set_index('race_ethnicity', inplace=True)
df_case_ethnicity.drop(labels=['White'], axis=0, inplace=True)
df_case_ethnicity.reset_index(inplace=True)
df_case_ethnicity = df_case_ethnicity.groupby('date').sum()
df_case_ethnicity.drop(labels=['percent_ca_population', 'deaths','case_percentage'], axis=1, inplace=True)
df_case_ethnicity['minority_new_positives'] = None
for i in range(1, len(df_case_ethnicity)):
    df_case_ethnicity.iloc[[i],[2]] = df_case_ethnicity.iloc[i,0] - df_case_ethnicity.iloc[(i-1),0]
df_case_ethnicity.rename(columns={'cases':'minority_cumulative_cases', 'death_percentage':'minority_proportion_deaths'},inplace=True)

#### Merge all dataframes together on the date index

In [29]:
df_list = [df_cases, df_hospitals, df_nursinghome, df_beds, df_case_age, df_case_ethnicity]
df = df_testing
for item in df_list:
    df = df.merge(item, how='outer', left_index=True, right_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 278 entries, 2020-03-18 to 2020-12-20
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   cumulative_tests                       277 non-null    float64
 1   new_tests                              277 non-null    float64
 2   cumulative_cases                       277 non-null    float64
 3   cumulative_deaths                      277 non-null    float64
 4   new_cases                              277 non-null    float64
 5   new_deaths                             277 non-null    float64
 6   hospitalized_covid_confirmed_patients  266 non-null    float64
 7   hospitalized_suspected_covid_patients  266 non-null    float64
 8   icu_covid_confirmed_patients           266 non-null    float64
 9   icu_suspected_covid_patients           266 non-null    float64
 10  icu_available_beds                     266 non-null    

In [30]:
df.to_csv('ca_covid_data.csv')